In [68]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model , Sequential
import os
from PIL import Image
import random
import os
import pickle
# from sklearn.model_selection import train_test_split

In [69]:
image_dimensions = {'height':256, 'width':256, 'channels':3}

In [70]:
x = []
y = []

In [ ]:
folder_path = "./Train/Fake"

c = 0 
for file in os.listdir(folder_path):
    if file.endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        image_path = os.path.join(folder_path, file)
        img = Image.open(image_path)
        img = img.resize((224,224))
        x.append(np.array(img))
        y.append(0)
        print(c)
        c=c+1
folder_path = "./Train/Real"


for file in os.listdir(folder_path):
    if file.endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        image_path = os.path.join(folder_path, file)
        img = Image.open(image_path)
        img = img.resize((224,224))
        x.append(np.array(img))
        y.append(1)
        print(c)
        c=c+1
       

In [72]:
print(len(x))

115576


In [73]:
combined = list(zip(x, y))
random.shuffle(combined)
x, y = zip(*combined)

In [74]:
x = np.array(x)

In [75]:
y= np.array(y)

In [76]:
x_train = x[:60000]
# x_test = x[75000:]
y_train = y[:60000]
# y_test = y[75000:]

In [77]:
print(type(x_train))

<class 'numpy.ndarray'>


In [78]:

print(y[0])

1


In [62]:
model = Sequential([
    Conv2D(8, (3, 3), padding='same', activation='relu', input_shape=(256, 256, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), padding='same'),

    Conv2D(8, (5, 5), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), padding='same'),

    Conv2D(16, (5, 5), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), padding='same'),

    Conv2D(16, (5, 5), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(4, 4), padding='same'),

    Flatten(),
    Dropout(0.5),
    Dense(16),
    LeakyReLU(alpha=0.1),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [63]:
model.compile(optimizer ='adam' ,loss ='binary_crossentropy' , metrics=['accuracy'])

In [64]:
model.fit(x_train,y_train , epochs=20)

Epoch 1/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.6598 - loss: 0.7097
Epoch 2/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.8009 - loss: 0.4339
Epoch 3/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.8589 - loss: 0.3257
Epoch 4/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.8909 - loss: 0.2608
Epoch 5/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.9027 - loss: 0.2324
Epoch 6/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9153 - loss: 0.2085
Epoch 7/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9209 - loss: 0.1980
Epoch 8/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9287 - loss: 0.1831
Epoch 9/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9276 - loss: 0.1814
Epoch 10/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9372 - loss: 0.1606
Epoch 11/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9338 - loss: 0.1613
Epoch 12/20
1563/1

In [89]:
x_test = x[60000:110000]
y_test = y[60000:110000]

In [66]:
model.evaluate(x_test, y_test)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9464 - loss: 0.1391


[0.14035390317440033, 0.9461399912834167]

In [67]:
model.save("image_detect.h5")

In [85]:
IMGWIDTH = 224

def InceptionLayer(x, a, b, c, d):
    x1 = Conv2D(a, (1, 1), padding='same', activation='relu')(x)
    
    x2 = Conv2D(b, (1, 1), padding='same', activation='relu')(x)
    x2 = Conv2D(b, (3, 3), padding='same', activation='relu')(x2)
    
    x3 = Conv2D(c, (1, 1), padding='same', activation='relu')(x)
    x3 = Conv2D(c, (3, 3), dilation_rate=2, strides=1, padding='same', activation='relu')(x3)
    
    x4 = Conv2D(d, (1, 1), padding='same', activation='relu')(x)
    x4 = Conv2D(d, (3, 3), dilation_rate=3, strides=1, padding='same', activation='relu')(x4)

    y = Concatenate(axis=-1)([x1, x2, x3, x4])
    
    return y

def MesoInception4(learning_rate=0.001):
    x = Input(shape=(IMGWIDTH, IMGWIDTH, 3))
    
    x1 = InceptionLayer(x, 1, 4, 4, 2)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
    
    x2 = InceptionLayer(x1, 2, 4, 4, 2)
    x2 = BatchNormalization()(x2)
    x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)        
    
    x3 = Conv2D(16, (5, 5), padding='same', activation='relu')(x2)
    x3 = BatchNormalization()(x3)
    x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
    
    x4 = Conv2D(16, (5, 5), padding='same', activation='relu')(x3)
    x4 = BatchNormalization()(x4)
    x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
    
    y = Flatten()(x4)
    y = Dropout(0.5)(y)
    y = Dense(16)(y)
    y = LeakyReLU(alpha=0.1)(y)
    y = Dropout(0.5)(y)
    y = Dense(1, activation='sigmoid')(y)
    
    model = Model(inputs=x, outputs=y)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

model = MesoInception4()



In [86]:
model.fit(x_train,y_train , epochs=20)

Epoch 1/20
   7/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 23ms/step - accuracy: 0.6181 - loss: 1.4188

I0000 00:00:1710686026.765818  601278 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_2', 4 bytes spill stores, 4 bytes spill loads



1875/1875 ━━━━━━━━━━━━━━━━━━━━ 39s 17ms/step - accuracy: 0.6592 - loss: 0.7173
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - accuracy: 0.8326 - loss: 0.3690
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - accuracy: 0.8904 - loss: 0.2556
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - accuracy: 0.9124 - loss: 0.2126
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 32s 17ms/step - accuracy: 0.9223 - loss: 0.1952
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - accuracy: 0.9288 - loss: 0.1790
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - accuracy: 0.9316 - loss: 0.1703
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - accuracy: 0.9378 - loss: 0.1570
Epoch 9/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - accuracy: 0.9389 - loss: 0.1548
Epoch 10/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - accuracy: 0.9444 - loss: 0.1404
Epoch 11/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - accuracy: 0.9459 - loss: 0.1382
Epoch 12/20
1875/18

In [88]:
model.save("m_icept.h5")

In [90]:
model.evaluate(x_test, y_test)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9571 - loss: 0.1093


[0.11077163368463516, 0.9568399786949158]